In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

In [39]:
customer_csv_file_path = (
    r"./data/customer_master.csv"
)
df_customer = pd.read_csv(customer_csv_file_path, encoding='cp949')

In [40]:
df_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


In [18]:
class_csv_file_path = (
    r"./data/class_master.csv"
)
df_class = pd.read_csv(class_csv_file_path)

In [19]:
df_class.head()

,class,class_name,price
0,C01,0_종일,10500
1,C02,1_주간,7500
2,C03,2_야간,6000


In [21]:
campaign_csv_file_path = (
    r"./data/campaign_master.csv"
)
df_campaign = pd.read_csv(campaign_csv_file_path)

In [22]:
df_campaign.head()

,campaign_id,campaign_name
0,CA1,2_일반
1,CA2,0_입회비반액할인
2,CA3,1_입회비무료


In [24]:
use_log_csv_file_path = (
    r"./data/use_log.csv"
)
df_use_log = pd.read_csv(use_log_csv_file_path)

In [25]:
df_use_log.head()

,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [26]:
# MySQL connection 세팅
db_user = "root"
db_password = "12341234"
db_host = "localhost"
db_port = "3306"
db_name = "churn_db"

In [27]:
# SQLAlchemy 엔진 생성 (데이터베이스 생성 전용)
create_engine_without_db = create_engine(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}"
)

In [28]:
# 데이터베이스 생성
with create_engine_without_db.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {db_name};"))

    # SQLAlchemy 엔진 생성
    engine = create_engine(
        f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
    )

In [42]:
try:
    with engine.connect() as conn:
        conn.execute(text(f"USE {db_name};"))

        table_names = ['customer_master', 'class_master', 'campaign_master', 'use_log']

        # 중복되지 않도록 테이블 DROP하기
        for i in range(len(table_names)):
            conn.execute((text)(f"DROP TABLE IF EXISTS {table_names[i]};"))

        create_customer_table = """
        CREATE TABLE IF NOT EXISTS customer_master(
            customer_id VARCHAR(20) PRIMARY KEY,
            name VARCHAR(20) NOT NULL,
            class VARCHAR(20) NOT NULL,
            gender varchar(5),
            start_date date NOT NULL,
            end_date date,
            campaign_id VARCHAR(20) NOT NULL,
            is_deleted boolean
        );
        """

        create_class_table = """
        CREATE TABLE IF NOT EXISTS class_master(
            class VARCHAR(20) NOT NULL,
            class_name VARCHAR(20) NOT NULL,
            price int NOT NULL
        );
        """
        create_campaign_table = """
        CREATE TABLE IF NOT EXISTS campaign_master(
            campaign_id VARCHAR(20) NOT NULL,
            campaign_name VARCHAR(20) NOT NULL
        );
        """
        create_use_log_table = """
        CREATE TABLE IF NOT EXISTS use_log(
            log_id VARCHAR(20) PRIMARY KEY,
            customer_id VARCHAR(20),
            usedate date NOT NULL
        );
        """

        # 테이블 만들기
        conn.execute(text(create_customer_table))
        conn.execute(text(create_class_table))
        conn.execute(text(create_campaign_table))
        conn.execute(text(create_use_log_table))

        # Dataframe.to_sql을 사용해서 데이터 넣기
        df_customer.to_sql(table_names[0], con=engine, index=False, if_exists="append", method="multi")
        df_class.to_sql(table_names[1], con=engine, index=False, if_exists="append", method="multi")
        df_campaign.to_sql(table_names[2], con=engine, index=False, if_exists="append", method="multi")
        df_use_log.to_sql(table_names[3], con=engine, index=False, if_exists="append", method="multi")
    
except Exception as e:
        print(f"Error: {e}")

# Connection 닫기
engine.dispose()